In [1]:
!pip install scikit-learn
!pip install opencv-python

In [2]:
import os
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
os.chdir('/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3')

In [4]:
import h5py
import numpy as np
import os

source_crop = [
    'cropped_real_300x300.h5',
    'cropped_synthetic_300x300.h5'
]

source_crop_1200 = [
    'cropped_real_1200x1200.h5',
    'cropped_synthetic_1200x1200.h5'
]

combine_crop_n1 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/n_1/combined_images_vds_300.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_300.h5'
]

combine_crop_n2 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/n_2/combined_images_vds_300.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_300.h5'
]

combine_crop_n3 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/n_3/combined_images_vds_300.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_300.h5'
]

combine_crop_n5 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/n_5/combined_images_vds_300.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_300.h5'
]

combine_crop_1200_n1 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_1/combined_images_vds_1200.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_1200.h5'
]

combine_crop_1200_n2 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_2/combined_images_vds_1200.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_1200.h5'
]

combine_crop_1200_n3 = [
    '/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/combined_images_vds_1200.h5',
    '/Volumes/T7/drive_thesis/thesis/img/converted/combined_images_vds_1200.h5'
]

source_1200 = [
    'ag_real_1200x1200.h5',
    'ag_synthetic_1200x1200.h5',
    'chen_real_1200x1200.h5',
    'chen_synthetic_1200x1200.h5'
]

source_600 = [
    'ag_real_600x600.h5',
    'ag_synthetic_600x600.h5',
    'chen_real_600x600.h5',
    'chen_synthetic_600x600.h5'
]

source_300 = [
    'ag_real_300x300.h5',
    'ag_synthetic_300x300.h5',
    'chen_real_300x300.h5',
    'chen_synthetic_300x300.h5'
]

source_200 = [
    'ag_real_200x200.h5',
    'ag_synthetic_200x200.h5',
    'chen_real_200x200.h5',
    'chen_synthetic_200x200.h5'
]

source_100 = [
    'ag_real_100x100.h5',
    'ag_synthetic_100x100.h5',
    'chen_real_100x100.h5',
    'chen_synthetic_100x100.h5'
]


source_h5_files = source_crop_1200

source_labels = [0, 1]
size = 1200

target_vds_h5_path = f'combined_images_vds_{size}.h5'
dataset_name_in_source = 'images'
label_dataset_name = 'labels'

In [5]:
batch_size = 64
total_images = 0
label_arrays = []
image_shape = None

print("Reading source file info and collecting labels...")
for i, source_file in enumerate(source_h5_files):
    try:
        with h5py.File(source_file, 'r') as hf_source:
            if dataset_name_in_source not in hf_source:
                print(f"Error: Dataset '{dataset_name_in_source}' not found in {source_file}")
                continue

            source_dataset = hf_source[dataset_name_in_source]
            num_images_in_file = source_dataset.shape[0]
            current_image_shape = source_dataset.shape[1:]

            print(f"  File: {source_file}, Images: {num_images_in_file}, Shape: {current_image_shape}, Dtype: {source_dataset.dtype}")

            if image_shape is None:
                image_shape = current_image_shape
            elif image_shape != current_image_shape:
                print(f"Warning: Image shape mismatch! Expected {image_shape}, found {current_image_shape} in {source_file}. VDS requires consistent image shapes.")

            label = source_labels[i]
            labels_for_file = np.full((num_images_in_file,), label, dtype=np.int64)
            label_arrays.append(labels_for_file)

            total_images += num_images_in_file

    except FileNotFoundError:
        print(f"Error: Source file not found: {source_file}")
    except Exception as e:
        print(f"An error occurred while processing {source_file}: {e}")


if label_arrays:
    y = np.concatenate(label_arrays, axis=0)
    print(f"\nSuccessfully concatenated labels. Total labels (y) shape: {y.shape}, Dtype: {y.dtype}")
else:
    y = np.array([], dtype=np.int64)
    print("\nNo label data collected.")


if total_images == 0 or image_shape is None:
    print("\nCannot create VDS: No images found or image shape is unknown.")
else:
    print(f"\nDefining VDS layout for total images: {total_images}, image shape: {image_shape}")

    vds_shape = (total_images,) + image_shape

    layout = h5py.VirtualLayout(shape=vds_shape, dtype=np.uint8)

    current_vds_offset = 0
    print("Mapping source datasets to VDS layout:")
    for i, source_file in enumerate(source_h5_files):
        try:
            with h5py.File(source_file, 'r') as hf_source:
                 if dataset_name_in_source not in hf_source:
                     continue

                 source_dataset = hf_source[dataset_name_in_source]
                 num_images_in_file = source_dataset.shape[0]

                 vsource = h5py.VirtualSource(
                     source_file,
                     dataset_name_in_source,
                     shape=source_dataset.shape
                 )

                 vds_slice = slice(current_vds_offset, current_vds_offset + num_images_in_file)

                 layout[vds_slice, ...] = vsource[0:num_images_in_file, ...]

                 print(f"  Mapped {source_file}[{dataset_name_in_source}] to VDS slice {vds_slice}")

                 current_vds_offset += num_images_in_file

        except FileNotFoundError:
            continue
        except Exception as e:
            print(f"An error occurred while defining VDS layout for {source_file}: {e}")


    print(f"\nCreating target HDF5 file with VDS: {target_vds_h5_path}")
    try:
        with h5py.File(target_vds_h5_path, 'w', libver='latest') as hf_target:
            hf_target.create_virtual_dataset('images_vds', layout)

            if y.size > 0:
                 hf_target.create_dataset(label_dataset_name, data=y)
                 print(f"Saved concatenated labels as dataset '{label_dataset_name}' in target file.")

            print(f"Successfully created target HDF5 file with VDS: {target_vds_h5_path}")

    except Exception as e:
        print(f"An error occurred while creating the target HDF5 file or VDS: {e}")

print("\n--- Demonstration of accessing data via VDS ---")
if os.path.exists(target_vds_h5_path):
    try:
        with h5py.File(target_vds_h5_path, 'r') as hf_vds:
            if 'images_vds' in hf_vds:
                vds_dataset = hf_vds['images_vds']

                print(f"Accessed Virtual Dataset 'images_vds'.")
                print(f"VDS Shape: {vds_dataset.shape}")
                print(f"VDS Dtype: {vds_dataset.dtype}")

                if vds_dataset.shape[0] > 0:
                     first_image_vds = vds_dataset[0]
                     print(f"  Read first image from VDS. Shape: {first_image_vds.shape}, Dtype: {first_image_vds.dtype}")

                batch_start = total_images // 2 if total_images > 0 else 0
                batch_end = batch_start + batch_size if total_images > 0 else 0

                if total_images > 0 and batch_start < total_images and batch_start < batch_end and batch_end <= total_images:
                    middle_batch_vds = vds_dataset[batch_start:batch_end]
                    print(f"  Read batch from VDS [{batch_start}:{batch_end}]. Shape: {middle_batch_vds.shape}, Dtype: {middle_batch_vds.dtype}")

            else:
                print("Error: 'images_vds' dataset not found in the target HDF5 file.")

            if label_dataset_name in hf_vds:
                loaded_labels_vds_file = hf_vds[label_dataset_name]
                print(f"Accessed concatenated labels dataset '{label_dataset_name}'.")
                print(f"Loaded labels shape: {loaded_labels_vds_file.shape}, Dtype: {loaded_labels_vds_file.dtype}")


    except FileNotFoundError:
        print(f"Error: Target VDS file not found: {target_vds_h5_path}")
    except Exception as e:
        print(f"An error occurred while accessing data from VDS: {e}")

else:
    print(f"\nTarget VDS file '{target_vds_h5_path}' was not created successfully.")


print("\n--- Verification ---")
if 'y' in locals():
    print(f"Final concatenated labels (y) shape: {y.shape}")
if os.path.exists(target_vds_h5_path):
    try:
        with h5py.File(target_vds_h5_path, 'r') as hf_vds:
             if 'images_vds' in hf_vds:
                 vds_dataset = hf_vds['images_vds']
                 print(f"Final Virtual Dataset (images_vds) shape: {vds_dataset.shape}")
    except Exception as e:
        print(f"Error verifying VDS shape: {e}")

Reading source file info and collecting labels...
  File: cropped_real_1200x1200.h5, Images: 3708, Shape: (1200, 1200, 3), Dtype: uint8
  File: cropped_synthetic_1200x1200.h5, Images: 920, Shape: (1200, 1200, 3), Dtype: uint8

Successfully concatenated labels. Total labels (y) shape: (4628,), Dtype: int64

Defining VDS layout for total images: 4628, image shape: (1200, 1200, 3)
Mapping source datasets to VDS layout:
  Mapped cropped_real_1200x1200.h5[images] to VDS slice slice(0, 3708, None)
  Mapped cropped_synthetic_1200x1200.h5[images] to VDS slice slice(3708, 4628, None)

Creating target HDF5 file with VDS: combined_images_vds_1200.h5
Saved concatenated labels as dataset 'labels' in target file.
Successfully created target HDF5 file with VDS: combined_images_vds_1200.h5

--- Demonstration of accessing data via VDS ---
Accessed Virtual Dataset 'images_vds'.
VDS Shape: (4628, 1200, 1200, 3)
VDS Dtype: uint8
  Read first image from VDS. Shape: (1200, 1200, 3), Dtype: uint8
  Read batc

In [7]:
def create_train_test_split_from_vds(
    hdf5_path,
    test_size,
    output_dir='.',
    random_state=None,
    image_dataset_name='images_vds',
    label_dataset_name='labels',
    output_h5_filename='train_test_split.h5',
    label_chunk_size=10000
):

    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    output_h5_path = os.path.join(output_dir, output_h5_filename)

    print(f"Attempting to load data from {hdf5_path} and save split to {output_h5_path}...")

    try:
        with h5py.File(hdf5_path, 'r') as hf_in:
            if image_dataset_name not in hf_in:
                print(f"Error: Image dataset '{image_dataset_name}' not found in {hf_in.filename}")
                return

            if label_dataset_name not in hf_in:
                 print(f"Error: Label dataset '{label_dataset_name}' not found in {hf_in.filename}")
                 return

            images_vds = hf_in[image_dataset_name]
            labels_dataset = hf_in[label_dataset_name]

            print(f"Successfully accessed '{image_dataset_name}' (Shape: {images_vds.shape}, Dtype: {images_vds.dtype})")
            print(f"Successfully accessed '{label_dataset_name}' (Shape: {labels_dataset.shape}, Dtype: {labels_dataset.dtype})")

            if images_vds.shape[0] != labels_dataset.shape[0]:
                print(f"Error: Number of images ({images_vds.shape[0]}) and labels ({labels_dataset.shape[0]}) do not match.")
                return

            total_samples = images_vds.shape[0]
            print(f"Total samples found: {total_samples}")


            print("Reading labels in chunks to determine class distribution for stratification...")
            label_counts = defaultdict(int)
            label_indices_by_class = defaultdict(list)

            for i in range(0, total_samples, label_chunk_size):
                end_idx = min(i + label_chunk_size, total_samples)
                label_chunk = labels_dataset[i:end_idx]
                current_indices = np.arange(i, end_idx)

                for label, index in zip(label_chunk, current_indices):
                    label_counts[label] += 1
                    label_indices_by_class[label].append(index)

            print("Label counts by class:", dict(label_counts))

            train_indices = []
            test_indices = []

            for label, indices in label_indices_by_class.items():
                if len(indices) > 0:
                    current_test_size = test_size
                    if isinstance(test_size, float):
                         current_test_size = max(1, int(len(indices) * test_size))
                    elif isinstance(test_size, int):
                         current_test_size = min(test_size, len(indices))

                    if current_test_size == len(indices):
                         print(f"Warning: All samples in class {label} ({len(indices)}) will be put in the test set as test_size is equal to class size.")
                         class_train_indices = []
                         class_test_indices = indices
                    elif current_test_size == 0:
                         print(f"Warning: No samples from class {label} will be in the test set as test_size is 0 or calculated test_size is 0.")
                         class_train_indices = indices
                         class_test_indices = []

                    elif len(indices) < current_test_size:
                         print(f"Warning: Not enough samples ({len(indices)}) in class {label} to meet requested test_size ({current_test_size}). All available will be used for split.")
                         class_train_indices, class_test_indices = train_test_split(
                            indices,
                            test_size=current_test_size,
                            random_state=random_state,
                            shuffle=True
                         )

                    else:
                        class_train_indices, class_test_indices = train_test_split(
                            indices,
                            test_size=current_test_size,
                            random_state=random_state,
                            shuffle=True
                        )

                    train_indices.extend(class_train_indices)
                    test_indices.extend(class_test_indices)
                else:
                    print(f"Warning: No samples found for class {label}.")

            train_indices = np.array(train_indices)
            test_indices = np.array(test_indices)

            np.random.shuffle(train_indices, random_state=42)
            np.random.shuffle(test_indices, random_state=42)


            print(f"Train indices shape: {train_indices.shape}")
            print(f"Test indices shape: {test_indices.shape}")

            print(f"\nSaving train/test sets to {output_h5_path}...")

            try:
                with h5py.File(output_h5_path, 'w') as hf_out:
                    print("Loading and saving training data...")
                    train_indices_sorted = np.sort(train_indices)
                    batch_size_save = 1000
                    for i in range(0, len(train_indices_sorted), batch_size_save):
                        end_idx_save = min(i + batch_size_save, len(train_indices_sorted))
                        current_indices_to_load = train_indices_sorted[i:end_idx_save]

                        X_train_batch = images_vds[current_indices_to_load]
                        y_train_batch = labels_dataset[current_indices_to_load]

                        if i == 0:
                             hf_out.create_dataset('X_train', data=X_train_batch, maxshape=(None,) + X_train_batch.shape[1:], dtype=X_train_batch.dtype)
                             hf_out.create_dataset('y_train', data=y_train_batch, maxshape=(None,), dtype=y_train_batch.dtype)
                        else:
                             hf_out['X_train'].resize((hf_out['X_train'].shape[0] + X_train_batch.shape[0]), axis=0)
                             hf_out['X_train'][-X_train_batch.shape[0]:] = X_train_batch

                             hf_out['y_train'].resize((hf_out['y_train'].shape[0] + y_train_batch.shape[0]), axis=0)
                             hf_out['y_train'][-y_train_batch.shape[0]:] = y_train_batch

                        print(f"  Saved batch {i//batch_size_save + 1}. Current X_train size: {hf_out['X_train'].shape[0]}")


                    print(f"Finished saving X_train (Shape: {hf_out['X_train'].shape}) and y_train (Shape: {hf_out['y_train'].shape})")


                    print("Loading and saving testing data...")
                    test_indices_sorted = np.sort(test_indices)
                    for i in range(0, len(test_indices_sorted), batch_size_save):
                         end_idx_save = min(i + batch_size_save, len(test_indices_sorted))
                         current_indices_to_load = test_indices_sorted[i:end_idx_save]

                         X_test_batch = images_vds[current_indices_to_load]
                         y_test_batch = labels_dataset[current_indices_to_load]

                         if i == 0:
                             hf_out.create_dataset('X_test', data=X_test_batch, maxshape=(None,) + X_test_batch.shape[1:], dtype=X_test_batch.dtype)
                             hf_out.create_dataset('y_test', data=y_test_batch, maxshape=(None,), dtype=y_test_batch.dtype)
                         else:
                             hf_out['X_test'].resize((hf_out['X_test'].shape[0] + X_test_batch.shape[0]), axis=0)
                             hf_out['X_test'][-X_test_batch.shape[0]:] = X_test_batch

                             hf_out['y_test'].resize((hf_out['y_test'].shape[0] + y_test_batch.shape[0]), axis=0)
                             hf_out['y_test'][-y_test_batch.shape[0]:] = y_test_batch

                         print(f"  Saved batch {i//batch_size_save + 1}. Current X_test size: {hf_out['X_test'].shape[0]}")

                    print(f"Finished saving X_test (Shape: {hf_out['X_test'].shape}) and y_test (Shape: {hf_out['y_test'].shape})")

                print("Train/test split successfully saved to HDF5.")

            except Exception as save_e:
                print(f"An error occurred while saving train/test sets to HDF5: {save_e}")


    except FileNotFoundError:
        print(f"Error: Input HDF5 file not found at {hdf5_path}")
    except Exception as e:
        print(f"An error occurred during train/test split process: {e}")

    print("Train/test split process finished.")

In [10]:
hdf5_path = f'/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/combined_images_vds_1200.h5'
test_size = 0.2
random_state = 42
output_dir = f'/Volumes/T7/drive_thesis/thesis/npy/cropped/1200/n_3'

create_train_test_split_from_vds(
    hdf5_path,
    test_size,
    output_dir=output_dir,
    random_state=random_state,
    image_dataset_name='images_vds',
    label_dataset_name='labels',
    output_h5_filename='train_test_split_crop.h5'
)

Attempting to load data from /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/combined_images_vds_1200.h5 and save split to /Volumes/T7/drive_thesis/thesis/npy/cropped/1200/n_3/train_test_split_crop.h5...
Successfully accessed 'images_vds' (Shape: (4628, 1200, 1200, 3), Dtype: uint8)
Successfully accessed 'labels' (Shape: (4628,), Dtype: int64)
Total samples found: 4628
Reading labels in chunks to determine class distribution for stratification...
Label counts by class: {np.int64(0): 3708, np.int64(1): 920}
Train indices shape: (3703,)
Test indices shape: (925,)

Saving train/test sets to /Volumes/T7/drive_thesis/thesis/npy/cropped/1200/n_3/train_test_split_crop.h5...
Loading and saving training data...
  Saved batch 1. Current X_train size: 1000
  Saved batch 2. Current X_train size: 2000
  Saved batch 3. Current X_train size: 3000
  Saved batch 4. Current X_train size: 3703
Finished saving X_train (Shape: (3703, 1200, 1200, 3)) and y_train (Shape: (3703,))
Loading and saving test